# 1.Trabajando con API's

La intención de este notebook es la de trabajar con API's, en este caso, para la descarga de información para su posterior analisis.

In [1]:
# Importando las librerias
import requests
import pandas as pd
import time

In [2]:
# Parametros para usar el API DE YT
# Amable recordatorio de siempre revisar esta pagina para crear la API}
# https://www.slickremix.com/docs/get-api-key-for-youtube/
# https://console.cloud.google.com/project
# XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

API_KEY = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
CHANNEL_ID = 'UCBNvm8XDKtQhoi6usy4HXAw'

In [3]:
# Prueba de la API
response = requests.get('https://api.github.com').json()

In [4]:
# Si se imprime información vamos por buen camino
response

{'current_user_url': 'https://api.github.com/user',
 'current_user_authorizations_html_url': 'https://github.com/settings/connections/applications{/client_id}',
 'authorizations_url': 'https://api.github.com/authorizations',
 'code_search_url': 'https://api.github.com/search/code?q={query}{&page,per_page,sort,order}',
 'commit_search_url': 'https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}',
 'emails_url': 'https://api.github.com/user/emails',
 'emojis_url': 'https://api.github.com/emojis',
 'events_url': 'https://api.github.com/events',
 'feeds_url': 'https://api.github.com/feeds',
 'followers_url': 'https://api.github.com/user/followers',
 'following_url': 'https://api.github.com/user/following{/target}',
 'gists_url': 'https://api.github.com/gists{/gist_id}',
 'hub_url': 'https://api.github.com/hub',
 'issue_search_url': 'https://api.github.com/search/issues?q={query}{&page,per_page,sort,order}',
 'issues_url': 'https://api.github.com/issues',
 'keys_url': '

## 1.1 Llamada a la API

La estructura de la URL para consultar la data de YT es la siguiente

'https://www.googleapis.com/youtube/v3/search?key='+**API_KEY**+"&channelId="+**CHANNEL_ID**+"&part=snippet,id&order=date&maxResults=10000"

* API_KEY es la key que se genero anteriormente

* CHANNEL_ID del canal objetivo

Para más información, consultar la documentación oficial: https://developers.google.com/youtube/v3/docs/search

In [5]:
# en este caso me interesa buscar la información del canal de Dross
# el CHANNEL_ID es el siguiente UCNYW2vfGrUE6R5mIJYzkRyQ
# en la parte de Results=10000 estamos pidiendo la informacion de los 10000 videos subidos (en donde espero que no exista gente loca que suba tantos videos)

channel_objetive = 'UCNYW2vfGrUE6R5mIJYzkRyQ'
pageToken = ''

url = 'https://www.googleapis.com/youtube/v3/search?key='+API_KEY+'&channelId='+channel_objetive+'&part=snippet,id&order=date&maxResults=10000'+pageToken

In [6]:
# Verificamos si hay respuesta con los datos necesarios
response = requests.get(url=url).json()

In [7]:
response.keys()

dict_keys(['kind', 'etag', 'nextPageToken', 'regionCode', 'pageInfo', 'items'])

In [8]:
# Se observa el formato JSON del ultimo video subido por Dross

response['items'][0]

{'kind': 'youtube#searchResult',
 'etag': 'AZCCJQECtSohwdHfr0k1zrSe3Yc',
 'id': {'kind': 'youtube#video', 'videoId': 'ulBV6QGcRrs'},
 'snippet': {'publishedAt': '2023-03-13T02:00:17Z',
  'channelId': 'UCNYW2vfGrUE6R5mIJYzkRyQ',
  'title': 'El HORRIBLE TRABAJO de Tim Saver',
  'description': 'Si el video te gusta, valoraría muchísimo que le dieras like y que comentaras algo. ¡Gracias! ❤ | ¡Suscríbete! http://bit.ly/1a1sm3k ...',
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/ulBV6QGcRrs/default.jpg',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i.ytimg.com/vi/ulBV6QGcRrs/mqdefault.jpg',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i.ytimg.com/vi/ulBV6QGcRrs/hqdefault.jpg',
    'width': 480,
    'height': 360}},
  'channelTitle': 'DrossRotzank',
  'liveBroadcastContent': 'none',
  'publishTime': '2023-03-13T02:00:17Z'}}

## 1.2 Seleccionando informacion

Apartir del JSON generado, es posible seleccionar determinada información, deacuerdo a las necesidades que se requieran

In [9]:
# Obtenemos el Id, titulo y fecha de subida del ultimo video publicado por Dross

video_id = response['items'][0]['id']['videoId']

video_title = response['items'][0]['snippet']['title']
video_title = str(video_title).replace('&amp;', '')

upload_date = response['items'][0]['snippet']['publishedAt']
upload_date = str(upload_date).split('T')[0]

print(f'''ID del video: {video_id}
Titulo del video: {video_title}
Fecha de subida: {upload_date}''')


ID del video: ulBV6QGcRrs
Titulo del video: El HORRIBLE TRABAJO de Tim Saver
Fecha de subida: 2023-03-13


## 1.3 Recolección de información

Creamos un dataframe vacio, pero con las columnnas a emplear

In [10]:
dataframe = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date',
                                  'view_count', 'like_count', 'comment_count'])

In [12]:
for info_video in response['items']:
    # Verificamos que todos los resultados sean videos
    if info_video['id']['kind'] == 'youtube#video':
        video_id = info_video['id']['videoId']

        video_title = info_video['snippet']['title']
        video_title = str(video_title).replace('&amp;', '')

        upload_date = info_video['snippet']['publishedAt']
        upload_date = str(upload_date).split('T')[0]
        
        url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
        response_video_stats = requests.get(url_video_stats).json()
        
        view_count = response_video_stats['items'][0]['statistics']['viewCount']
        like_count = response_video_stats['items'][0]['statistics']['likeCount']
        comment_count = response_video_stats['items'][0]['statistics']['commentCount']
        
        column = pd.DataFrame([[video_id, video_title, upload_date,
                               view_count, like_count, comment_count]],
                              columns=['video_id', 'video_title', 'upload_date',
                                       'view_count', 'like_count', 'comment_count'])
        
        dataframe = pd.concat([dataframe, column])
        

print(f'''ID del video: {video_id}
Titulo del video: {video_title}
Fecha de subida: {upload_date}
Vistas: {view_count}
Likes: {like_count}
Comentarios: {comment_count}\n''')

dataframe.head()


ID del video: hODEtVXqn7s
Titulo del video: TOP: Los 7 asesinos seriales que hicieron los dibujos y pinturas más aterradores
Fecha de subida: 2022-10-24
Vistas: 1980420
Likes: 131802
Comentarios: 3808



,video_id,video_title,upload_date,view_count,like_count,comment_count
0,ulBV6QGcRrs,El HORRIBLE TRABAJO de Tim Saver,2023-03-13,872564,73253,4110
0,ttYiE_rnKyQ,Lorenzo JAMÁS PERDONARÁ a su madre,2023-03-09,1065424,86573,7331
0,imwnodeMW8Y,3 videos de cosas extrañas mirando a través de...,2023-03-07,1450537,104702,4789
0,pH5CWlIBoXM,El hoyo de Aleksandr,2023-03-04,1824018,153246,11195
0,eyrPKOcPXJw,Los 7 dibujos más escalofriantes y graciosos h...,2023-03-02,158163,9143,140


Usamos el ID del video más reciente

In [14]:
video_id = 'ulBV6QGcRrs'

In [51]:
url = "https://youtube.googleapis.com/youtube/v3/videos?part=snippet&id="+video_id+'&key='+API_KEY
response = requests.get(url=url).json()

In [40]:
url = "https://youtube.googleapis.com/youtube/v3/videos?part=snippet&id="+video_id+'&key='+API_KEY
response = requests.get(url=url).json()

video_title = response['items'][0]['snippet']['title']
video_title = str(video_title).replace('&amp;', '')

upload_date = response['items'][0]['snippet']['publishedAt']
upload_date = str(upload_date).split('T')[0]

tags = response['items'][0]['snippet']['tags']

url = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
response = requests.get(url=url).json()

view_count = response['items'][0]['statistics']['viewCount']
like_count = response['items'][0]['statistics']['likeCount']
comment_count = response['items'][0]['statistics']['commentCount']

print(f'Video ID: {video_id}')
print(f'Titulo del video: {video_title}')
print(f'Fecha de subida {upload_date}')
print(f'Cantidad de vistas: {view_count}')
print(f'Cantidad de likes: {like_count}')
print(f'Cantidad de comentarios {comment_count}')
print(f'Cantidad etiquetas {len(tags)}')

#print(response['items'][0]['snippet']['title'])
#print(response['items'][0]['snippet']['publishedAt'])

Video ID: ulBV6QGcRrs
Titulo del video: El HORRIBLE TRABAJO de Tim Saver
Fecha de subida 2023-03-13
Cantidad de vistas: 879926
Cantidad de likes: 73531
Cantidad de comentarios 4121
Cantidad etiquetas 27


In [49]:
# Esta función se toma por separado, ya que hace otro llamado a la API

def get_video_statistics(video_id):
    url = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
    
    response = requests.get(url=url).json()
    
    # return pd.DataFrame([[response['items'][0]['statistics']]],
    #                     columns=['video_id', 'video_title', 'upload_date',
    #                                    'view_count', 'like_count', 'comment_count'])
    
    return response['items'][0]['statistics']['viewCount'],\
            response['items'][0]['statistics']['likeCount'], \
            response['items'][0]['statistics']['commentCount']


In [50]:
data = get_video_statistics('ulBV6QGcRrs')
data

('893636', '73969', '4170')

In [61]:
url = 'https://www.googleapis.com/youtube/v3/search?key='+API_KEY+'&channelId='+\
            'UCC0a-WbVbyxXdgyB5Ki7jXA'+'&part=snippet,id&order=date&maxResults=10000'+pageToken
            
response = requests.get(url=url).json()


In [75]:
# Falta agregar la parte de las etiquetas
# Arreglar el Bug
# Solucionar la parte del append por concat

def get_video_info(df: pd.DataFrame, channel_ID):
    pageToken = ''
    while True:
        url = 'https://www.googleapis.com/youtube/v3/search?key='+API_KEY+'&channelId='+\
            channel_ID+'&part=snippet,id&order=date&maxResults=10000'+pageToken
            
        response = requests.get(url=url).json()
        time.sleep(1)
        
        for video in response['items']:
            if video['id']['kind'] == 'youtube#video':
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                upload_date = str(video['snippet']['publishedAt']).split('T')[0]
                view_count, like_count, comment_count = get_video_statistics(video_id)
                
                df = df.append({'Video_id': video_id, 'Video_title': video_title,
                                'Upload_date': upload_date, 'View_Count': view_count,
                                'Like_Count': like_count, 'Comment_Count': comment_count}, ignore_index=True)
                
        try:
            if response['nextPageToken'] != None:
                pageToken = 'pageToken=' + response['nextPageToken']
                
        except:
            break
        
    return df
                
        

In [76]:
df = pd.DataFrame(columns=["Video_id","Video_title","Upload_date",
"View_count","Like_count","Comment_count"]) #build our 
 
df = get_video_info(df, 'UCC0a-WbVbyxXdgyB5Ki7jXA')
 
print(df)

/tmp/ipykernel_251/3806624572.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Video_id': video_id, 'Video_title': video_title,
/tmp/ipykernel_251/3806624572.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Video_id': video_id, 'Video_title': video_title,
/tmp/ipykernel_251/3806624572.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Video_id': video_id, 'Video_title': video_title,
/tmp/ipykernel_251/3806624572.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Video_id': video_id, 'Video_title': video_title,
/tmp/ipykernel_251/3806624572.py:19: FutureWarni

KeyError: 'items'